In [16]:
from py2sambvca.py2sambvca import py2sambvca as p2s

input_xyz_path = 'xyz_structures//IRMOF-1.xyz'
sambvcaX_path = 'Sambvca21_source//sambvca21.x'

buried_vol = p2s(
    xyz_filepath=input_xyz,
    sphere_center_atom_ids=[],
    z_ax_atom_ids=[],
    xz_plane_atoms_ids=[],
    path_to_sambvcax=sambvcaX_path
    )

buried_vol.write_input()
buried_vol.calc()

True